In [1]:
!pip install opensearch-py

     |████████████████████████████████| 266kB 22.7MB/s eta 0:00:01
     |████████████████████████████████| 163kB 85.5MB/s eta 0:00:01
     |████████████████████████████████| 71kB 24.6MB/s eta 0:00:01
     |████████████████████████████████| 153kB 59.4MB/s eta 0:00:01
     |████████████████████████████████| 235kB 74.8MB/s eta 0:00:01
     |████████████████████████████████| 143kB 68.9MB/s eta 0:00:01
     |████████████████████████████████| 71kB 23.8MB/s eta 0:00:01
ERROR: openapi-schema-validator 0.6.2 has requirement jsonschema<5.0.0,>=4.19.1, but you'll have jsonschema 4.19.0 which is incompatible.
ERROR: mosaic-ai-client 1.0.0 has requirement matplotlib==3.1.1, but you'll have matplotlib 3.6.0 which is incompatible.
ERROR: jsonschema-path 0.3.2 has requirement referencing<0.32.0,>=0.28.0, but you'll have referencing 0.33.0 which is incompatible.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
from opensearchpy import OpenSearch
host = 'vpc-aw-fdc-opensearch-qa-01-klbmt4ru7oksvsemnnmjpohfw4.us-east-1.es.amazonaws.com'
port = 443
auth = ('qa-developer', 'wF5341W8=~') # For testing only. Don't store credentials in code.
#ca_certs_path = '/full/path/to/root-ca.pem' # Provide a CA bundle if you use intermediate CAs with your root CA.

# Create the client with SSL/TLS enabled, but hostname verification disabled.
client = OpenSearch(
    hosts = [{'host': host, 'port': port}],
    http_compress = True, # enables gzip compression for request bodies
    http_auth = auth,
    use_ssl = True,
    verify_certs = True,
    ssl_assert_hostname = False,
    ssl_show_warn = False
)

In [11]:
query = {
  "sort" : [
        { "@timestamp" : {"order" : "desc"}}
  ],
  "query": {
    "bool": {
        "must": [
            {"match_phrase" : {"kubernetes.labels.app": "fosfor-gatekeeper"}},
            {"regexp": {"message": "[0-9]{1,3}.[0-9]{1,3}.[0-9]{1,3}.[0-9]{1,3}"}}
        ],
        "filter" : { "bool" :   {   
            "must_not": [{"regexp" : {"message": "elb"}}]
            }
        }
      }
    }
}
    
response = client.search(
    body = query,
    index = 'filebeat-qa-*'
)

response

{'took': 569,
 'timed_out': False,
 '_shards': {'total': 15, 'successful': 15, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 10000, 'relation': 'gte'},
  'max_score': None,
  'hits': [{'_index': 'filebeat-qa-7.10.0-2024.05.23',
    '_id': 'tpd-pY8BhXLWb6NES9XS',
    '_score': None,
    '_source': {'@timestamp': '2024-05-23T12:19:23.567Z',
     'agent': {'id': 'ab35a026-cc83-41c4-98a3-cf75efff8c27',
      'name': 'filebeat-filebeat-9qgsb',
      'type': 'filebeat',
      'version': '7.10.0',
      'hostname': 'filebeat-filebeat-9qgsb',
      'ephemeral_id': '6279dd00-dd80-4fbb-b866-15f2835d5845'},
     'log': {'offset': 5974299,
      'file': {'path': '/var/log/containers/fosfor-gatekeeper-6c45fdb7cb-756jv_fdc-components_fosfor-gatekeeper-2508382f0020ea4f9d171f54acd9b2c68c9f08b709b5102346e2bc9a5a607742.log'}},
     'stream': 'stdout',
     'message': '10.229.14.228 - - [23/May/2024:12:19:23 +0000] "GET /analytics-config/v1/dataset/datasetId_Not%20Found/kpi?sort=_modified_date

In [4]:
count=0
for hit in response['hits']['hits']:
    count=count+1
    #print(hit["_source"]["message"])
print(count)

10
